## Use this notebook to take water bodies from the HydroPolys layer and add them to the Named_Reservoirs shapefile

In [1]:
%matplotlib inline

import sys

sys.path.append('Scripts')
from datahandling import wofs_fuser

import datacube
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path
from datacube.utils import masking
from datacube.utils import geometry 
from datacube.utils.geometry import CRS
from datacube.helpers import write_geotiff
import geopandas as gpd
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json



dc = datacube.Datacube()

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Read and fomat the polygon you want to add to the Named_Reservoirs shapefile

In [2]:
gdf = gpd.read_file('HydroPolys/GLENMAGGIE.shp')
gdf

,FEATURETYP,TYPE,NAME,PERENNIALI,HIERARCHY,DIMENSION,FEATUREREL,FEATURESOU,ATTRIBUTER,ATTRIBUTES,PLANIMETRI,GAID,AUSHYDRO_I,STKEHDRID,STKEHDRNAM,SHAPE_Leng,SHAPE_Area,NATURE,WATERSTORA,geometry
0,Lake,6,LAKE GLENMAGGIE,Perennial,None,17034147.0,1999-01-01,DSE,1999-01-01,DSE,20,3024303,12827698,40298827,DSE,0.550671,0.001746,None,None,"POLYGON ((146.72768 -37.85241, 146.72792 -37.8..."


In [3]:
#Get the centroid location of the reservoir

gdf = gdf.to_crs(epsg=4326) #4326 is the tag for lat lon format
#we need to get the lat and lon of where the dam is to call the right satellite data later
#we want the lat and lon to be columns in the data to make it easy when we query the wofs data later
#Getting the centroid from the polygon and adding a latitude and longitude column (I found this code on the internet):
def getXY(pt):
    return (pt.x, pt.y)
centroidseries = gdf['geometry'].centroid
x,y = [list(t) for t in zip(*map(getXY, centroidseries))]

gdf = gdf.assign(lon=x)
gdf = gdf.assign(lat=y)
gdf = gdf[['NAME', 'SHAPE_Leng', 'SHAPE_Area', 'lat', 'lon', 'geometry']]
gdf

/env/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  if __name__ == '__main__':


,NAME,SHAPE_Leng,SHAPE_Area,lat,lon,geometry
0,LAKE GLENMAGGIE,0.550671,0.001746,-37.906605,146.768497,"POLYGON ((146.72768 -37.85241, 146.72792 -37.8..."


In [4]:
csv = 'Library/GLENMAGGIE_225225A.csv'
gauge_number_df = pd.read_csv(csv, nrows=3, escapechar='#')
column2 = list(gauge_number_df)[1]
gauge_number_df = gauge_number_df.rename(columns = {column2 : 'gauge_ID'})
ID_str = gauge_number_df.at[0, 'gauge_ID']
lat = gauge_number_df.at[1, 'gauge_ID']
lon = gauge_number_df.at[2, 'gauge_ID']
gdf['gauge_ID'] = ID_str
gdf['lat_gauge'] = lat
gdf['lon_gauge'] = lon
gdf['Area_calc'] = ''
gdf['staion_nam'] = column2
gdf = gdf[['NAME', 'SHAPE_Leng', 'SHAPE_Area', 'lat', 'lon', 'Area_calc', 'staion_nam', 'gauge_ID', 'lat_gauge', 'lon_gauge', 'geometry']]
gdf

,NAME,SHAPE_Leng,SHAPE_Area,lat,lon,Area_calc,staion_nam,gauge_ID,lat_gauge,lon_gauge,geometry
0,LAKE GLENMAGGIE,0.550671,0.001746,-37.906605,146.768497,,MACALISTER RIVER @ LAKE GLENMAGGIE (HEAD GAUGE),225225A,-37.90833333,146.8,"POLYGON ((146.72768 -37.85241, 146.72792 -37.8..."


In [5]:
gdf2 = gpd.read_file('Named_Reservoirs/Named_Reservoirs5.shp')
gdf2 = gdf2.to_crs(epsg=4326)
gdf2

,NAME,SHAPE_Leng,SHAPE_Area,lat,lon,Area_calc,staion_nam,gauge_ID,lat_gauge,lon_gauge,geometry
0,WILSON RESERVOIR,0.034915,0.000028,-37.510018,144.018549,270561.736,LAL LAL C WILSON RES,232220A,-37.514400,144.017700,"POLYGON ((144.01821 -37.51455, 144.01719 -37.5..."
1,SUGARLOAF RESERVOIR,0.212903,0.000438,-37.672015,145.305467,4284887.270,SUGARLOAF RES DAM,229607A,-37.672400,145.279000,"POLYGON ((145.31084 -37.68275, 145.31059 -37.6..."
2,SUGARLOAF RESERVOIR,0.212903,0.000438,-37.672015,145.305467,4284887.270,SUGARLOAF RES. H.G.,408217A,-37.103981,143.402094,"POLYGON ((145.31084 -37.68275, 145.31059 -37.6..."
3,GREEN LAKE,0.155045,0.000715,-36.435338,144.834187,7111145.660,Greens Lake,sp-o10454,-36.433798,144.834474,"POLYGON ((144.82094 -36.43662, 144.82081 -36.4..."
4,GREEN LAKE,0.075718,0.000170,-36.792870,142.292810,1688188.340,Green Lake,GREEN,-36.792250,142.292100,"POLYGON ((142.29212 -36.80000, 142.29128 -36.7..."
...,...,...,...,...,...,...,...,...,...,...,...
420,CLAUDE WHARTON WEIR,0.163420,0.000133,-25.617167,151.559284,1481333.550,Claude Wharton HW,136003C,-25.614740,151.595000,"POLYGON ((151.52507 -25.62287, 151.52643 -25.6..."
421,KIRAR WEIR,0.257613,0.000240,-25.299279,151.063109,2681688.020,Kirar Weir HW,136121A,-25.345560,151.079067,"POLYGON ((151.07836 -25.34552, 151.07834 -25.3..."
422,MARIAN WEIR,0.266732,0.000134,-21.146641,148.885758,1544632.180,Marian Weir HW,125003A,-21.141222,148.936264,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
423,MARIAN WEIR,0.266732,0.000134,-21.146641,148.885758,1544632.180,Mirani Weir HW,125008A,-21.177880,148.830000,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."


In [6]:
gdf3 = gdf2.append(gdf)
gdf3

,NAME,SHAPE_Leng,SHAPE_Area,lat,lon,Area_calc,staion_nam,gauge_ID,lat_gauge,lon_gauge,geometry
0,WILSON RESERVOIR,0.034915,0.000028,-37.510018,144.018549,270562,LAL LAL C WILSON RES,232220A,-37.5144,144.018,"POLYGON ((144.01821 -37.51455, 144.01719 -37.5..."
1,SUGARLOAF RESERVOIR,0.212903,0.000438,-37.672015,145.305467,4.28489e+06,SUGARLOAF RES DAM,229607A,-37.6724,145.279,"POLYGON ((145.31084 -37.68275, 145.31059 -37.6..."
2,SUGARLOAF RESERVOIR,0.212903,0.000438,-37.672015,145.305467,4.28489e+06,SUGARLOAF RES. H.G.,408217A,-37.104,143.402,"POLYGON ((145.31084 -37.68275, 145.31059 -37.6..."
3,GREEN LAKE,0.155045,0.000715,-36.435338,144.834187,7.11115e+06,Greens Lake,sp-o10454,-36.4338,144.834,"POLYGON ((144.82094 -36.43662, 144.82081 -36.4..."
4,GREEN LAKE,0.075718,0.000170,-36.792870,142.292810,1.68819e+06,Green Lake,GREEN,-36.7923,142.292,"POLYGON ((142.29212 -36.80000, 142.29128 -36.7..."
...,...,...,...,...,...,...,...,...,...,...,...
421,KIRAR WEIR,0.257613,0.000240,-25.299279,151.063109,2.68169e+06,Kirar Weir HW,136121A,-25.3456,151.079,"POLYGON ((151.07836 -25.34552, 151.07834 -25.3..."
422,MARIAN WEIR,0.266732,0.000134,-21.146641,148.885758,1.54463e+06,Marian Weir HW,125003A,-21.1412,148.936,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
423,MARIAN WEIR,0.266732,0.000134,-21.146641,148.885758,1.54463e+06,Mirani Weir HW,125008A,-21.1779,148.83,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
424,TINANA BARRAGE,0.034002,0.000012,-25.571684,152.713346,129236,Tinana Barrage HW,138008A,-25.5721,152.718,"POLYGON ((152.71974 -25.56929, 152.71973 -25.5..."


In [7]:
gdf3.to_file('Named_Reservoirs7.shp')